In [ ]:
import os, sys, asyncio
from pathlib import Path
import nest_asyncio

# 修改为你的仓库路径
repo_root = Path("/content/cv_project")
sys.path.insert(0, str(repo_root / "src"))
os.environ["PYTHONPATH"] = str(repo_root / "src")

# 允许在已运行的 event loop 中再次 run_until_complete
nest_asyncio.apply()

from core.config import SystemConfig
from pipeline.video_semantic_search import VideoSemanticSystem

config = SystemConfig(
    video_path=repo_root / "data/raw/semantic/MOT17-12.mp4",
    output_dir=repo_root / "output/demo_run",
    vlm_backend="vllm",
    vllm_endpoint="http://localhost:8000/v1",
    vllm_model_name="Qwen/Qwen3-VL-4B-Instruct",
    router_backend="vllm",
)

question = "找穿蓝色衣服的人"

system = VideoSemanticSystem(config=config)
# 重写 _run_coroutine，避免 asyncio.run 冲突
system._run_coroutine = lambda coro: asyncio.get_event_loop().run_until_complete(coro)

print("=== Building index ===")
system.build_index()
print(f"Evidence packages: {len(system.evidence_map)}")

print(f"=== Running query: {question} ===")
results = system.question_search(question, top_k=3)
print(f"Matches: {len(results)}")
for r in results:
    print(f"track {r.track_id}: {r.start_s:.1f}s–{r.end_s:.1f}s | score={r.score:.2f} | {r.reason}")

print("Outputs in output/demo_run/: tracking_<question>.mp4 and tracking_all_tracks_<question>.mp4")
